In [ ]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="3"

from cnn_model import *
from matplotlib import pyplot as plt

########################################
from PIL import Image


#read image

########################################

'''
tf.reset_default_graph()
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('my_net/save_net.ckpt.meta')
    saver.restore(sess, tf.train.latest_checkpoint('my_net/'))
    sess.run(tf.global_variables_initializer())
    all_vars = tf.trainable_variables()
    for v in all_vars:
        print("%s with value %s" % (v.name, sess.run(v)))
'''
def details(model, sess):
    print("{:30}:{}".format("convolution weight 1", sess.run(model.W_conv1)))
    #print("{:30}:{}".format("convolution bias 1", sess.run(model.b_conv1)))
    print("{:30}:{}".format("convolution weight 2", sess.run(model.W_conv2)))
    #print("{:30}:{}".format("convolution bias 2", sess.run(model.b_conv2)))

    print("{:30}:{}".format("full connected weight 1", sess.run(model.W_fc1)))
    #print("{:30}:{}".format("full connected bias 1", sess.run(model.b_fc1)))
    print("{:30}:{}".format("full connected weight 2", sess.run(model.W_fc2)))
    #print("{:30}:{}".format("full connected bias 2", sess.run(model.b_fc2)))
    return


#load model
load_model=mnist_cnn_model()
export_dir = "./my_net/save_net.ckpt"
new_saver = tf.train.Saver({
            "W_conv1": load_model.W_conv1, 
            "W_conv2": load_model.W_conv2, 
            "b_conv1": load_model.b_conv1, 
            "b_conv2": load_model.b_conv2, 
            "W_fc1": load_model.W_fc1, 
            "W_fc2": load_model.W_fc2, 
            "b_fc1": load_model.b_fc1, 
            "b_fc2": load_model.b_fc2
        })
#saver = tf.train.Saver()
with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     new_saver = tf.train.import_meta_graph('my_net/save_net.ckpt-200.meta')
    new_saver.restore(sess, export_dir)#這裡一定有問題
    print("Model Restore form", export_dir)


    test_model = mnist_cnn_model()
    test_model.set_value(load_model.W_conv1, load_model.W_conv2, load_model.b_conv1, load_model.b_conv2, 
                load_model.W_fc1, load_model.W_fc2, load_model.b_fc1, load_model.b_fc2)

    predict_y = test_model.run_cnn()

    #show label that the most likely to the image
    correct_prediction = tf.equal(tf.argmax(predict_y,1), tf.argmax(test_model.label_placeholder,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


    img_test = mnist.test.images
    label_test = mnist.test.labels

    print(sess.run(predict_y, feed_dict={test_model.img_placeholder: img_test,
                                                       test_model.label_placeholder: label_test,
                                                       test_model.keep_prob: 1.0}))

    print("Accuracy: ", sess.run(accuracy, feed_dict={test_model.img_placeholder: img_test,
                                                       test_model.label_placeholder: label_test,
                                                       test_model.keep_prob: 1.0}))

    print("test:", sess.run(tf.argmax(mnist.test.labels,1)))
    print("model:", sess.run(tf.argmax(predict_y, 1), feed_dict={test_model.img_placeholder: img_test,
                                                                 test_model.label_placeholder: label_test, 
                                                                 test_model.keep_prob: 1.0}))

print(predict_y)

INFO:tensorflow:Restoring parameters from ./my_net/save_net.ckpt
Model Restore form ./my_net/save_net.ckpt


In [ ]:
# import the inspect_checkpoint library
from tensorflow.python.tools import inspect_checkpoint as chkp

# print all tensors in checkpoint file
chkp.print_tensors_in_checkpoint_file("my_net/save_net.ckpt", tensor_name='', all_tensors=True)